## 1. Carrega as bases de dados

In [22]:
import pandas as pd

PASTA_DADOS = './dados/'

# A pasta dos JURIS aqui não é a pasta original, e sim o resultado do caderno 1
PASTA_JURIS_TCU = f'{PASTA_DADOS}outputs/1_tratamento_juris_tcu/'

PASTA_RESULTADO_CADERNO = f'{PASTA_DADOS}outputs/7_gera_embeddings_termos/'

# Substituir embeddings já criados
SOBRESCREVER_EMBEDDINGS = False

# Tamanho do lote
TAMANHO_DO_LOTE = 150

# Carrega os arquivos 
def carrega_juris_tcu():
    doc1 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_1.csv', sep='|')
    doc2 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_2.csv', sep='|')
    doc3 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_3.csv', sep='|')
    doc4 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_4.csv', sep='|')
    doc = pd.concat([doc1, doc2, doc3, doc4], ignore_index=True)
    query = pd.read_csv(f'{PASTA_JURIS_TCU}query_tratado.csv', sep='|')
    qrel = pd.read_csv(f'{PASTA_JURIS_TCU}qrel_tratado.csv', sep='|')

    return doc, query, qrel

In [7]:
from formatador import remove_html

# Carrega as queries para query
doc, query, qrel = carrega_juris_tcu()

#Transforma dataframe em dicionário
query = query.to_dict(orient='list')
print(query.keys())

dict_keys(['KEY', 'TEXT', 'SOURCE'])


## 2. Tokenização das queries.

In [10]:
from transformers import AutoTokenizer

#Carregando o nokenizador Legal BERTimbau V3
model_ckpt = 'rufimelo/Legal-BERTimbau-sts-large-ma-v3'

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [11]:
#Definição da função que realizará a tokenização em lotes
def tokenize(batch):
    return tokenizer(batch["TEXT"], padding=True, truncation=True, return_tensors='pt', max_length=512)

In [12]:
# Aplicando a função de tokenização aos dados
query_encoded = query.copy()
tokenized_outputs = tokenize(query_encoded)
tokenized_outputs

{'input_ids': tensor([[  101,  4204,   122,  ...,     0,     0,     0],
        [  101,  8197,   123,  ...,     0,     0,     0],
        [  101,   602, 16166,  ...,     0,     0,     0],
        ...,
        [  101,   807,   441,  ...,     0,     0,     0],
        [  101, 17911, 22287,  ...,     0,     0,     0],
        [  101, 13082,   253,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [13]:
# Armazenando input_ids, attention_mask e token_type_ids em query_encoded
query_encoded['input_ids'] = tokenized_outputs['input_ids']
query_encoded['attention_mask'] = tokenized_outputs['attention_mask']
query_encoded['token_type_ids'] = tokenized_outputs['token_type_ids']
query_encoded.keys()

dict_keys(['KEY', 'TEXT', 'SOURCE', 'input_ids', 'attention_mask', 'token_type_ids'])

In [15]:
# Verifica se a tokenização foi realizada adequadamente
from itertools import islice

for termo, input_id in islice(zip(query_encoded["TEXT"], query_encoded["input_ids"]), 3):
    print(f"Termo: {termo}, \nInput IDs: {tokenizer.convert_ids_to_tokens(input_id)}\n")

Termo: técnica e preço, 
Input IDs: ['[CLS]', 'técnica', 'e', 'preço', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']

Termo: restos a pagar, 
Input IDs: ['[CLS]', 'restos', 'a', 'pagar', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']

Termo: aditivo a contrato, 
Input IDs: ['[CLS]', 'ad', '##itivo', 'a', 'contrato', '[SEP]', '[PAD]', '

## 3. Obtenção dos embeddings.

In [16]:
import torch
from transformers import AutoModel

# Carrega modelo

#Caso exista GPU utilize-a, caso contrário use a CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

In [17]:
# Função para agregaçção da última camada oculta pela média
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [18]:
# Função para extração da última camada oculta
def extract_hidden_states(batch):
    # Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items() 
              if k in tokenizer.model_input_names}
    # Extract last hidden states
    with torch.no_grad():
        model_output = model(**inputs)
    
    return batch['KEY'], model_output.last_hidden_state[:,0].cpu().numpy(), mean_pooling(model_output, inputs['attention_mask']).cpu().numpy()

In [19]:
# Função para dividir dicionário em lotes
def dividir_dicionario_em_lotes(dicionario, tamanho_do_lote):
    vetor_de_dicionarios = []
    max_len = max(len(v) for v in dicionario.values())  # Encontrando o vetor de valores mais longo
    
    for i in range(0, max_len, tamanho_do_lote):
        novo_dicionario = {chave: valores[i:i + tamanho_do_lote] for chave, valores in dicionario.items()}
        vetor_de_dicionarios.append(novo_dicionario)
    
    return vetor_de_dicionarios

In [20]:
import numpy as np

# Função para reconstruir dicionário a partir de lotes
def reconstruir_dicionario_a_partir_de_lotes(vetor_de_dicionarios):
    dicionario_reconstruido = {}
    
    # Inicializando listas vazias para cada chave no primeiro dicionário do vetor
    for chave in vetor_de_dicionarios[0].keys():
        dicionario_reconstruido[chave] = []
    
    # Iterando sobre cada dicionário no vetor e concatenando os valores para cada chave
    for dicionario_lote in vetor_de_dicionarios:
        for chave, valores in dicionario_lote.items():
            dicionario_reconstruido[chave].extend(valores)
    
    # Transforma numpy array em tensor
    dicionario_reconstruido['cls_hidden_state'] = torch.tensor(np.array(dicionario_reconstruido['cls_hidden_state']))
    dicionario_reconstruido['mean_hidden_state'] = torch.tensor(np.array(dicionario_reconstruido['mean_hidden_state']))
    
    return dicionario_reconstruido

In [36]:
# Processar e salvar embeddings
import pickle
from tqdm import tqdm
import os

# Divide query_encoded em lotes
query_encoded_em_lotes = dividir_dicionario_em_lotes(query_encoded, TAMANHO_DO_LOTE)

# Processa e salva embeddings
for i, dicionario in enumerate(tqdm(query_encoded_em_lotes), start=1):
    
    caminho_arquivo = f'{PASTA_RESULTADO_CADERNO}embeddings_query_{i}.pickle'
    if  not SOBRESCREVER_EMBEDDINGS and os.path.exists(caminho_arquivo):
        continue

    key, cls_hidden_state, mean_hidden_state = extract_hidden_states(dicionario)
    
    # Cria estrutura que será salva em arquivo
    embeddings_query = {
        'key': key,
        'cls_hidden_state': cls_hidden_state,
        'mean_hidden_state': mean_hidden_state
    }
    
    # Gravando lote em um arquivo .pickle
    with open(caminho_arquivo, 'wb') as arquivo_pickle:
        pickle.dump(embeddings_query, arquivo_pickle)

100%|██████████| 1/1 [00:00<?, ?it/s]


In [24]:
dicionario['input_ids'].size()
#dicionario['attention_mask']
#dicionario['token_type_ids']

torch.Size([150, 46])

In [25]:
query_encoded_em_lotes[0]['KEY']

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150]

## 4. Cálculo da distância entre os embeddings.

In [38]:
# Função para restaurar embeddings dos arquivos pickle

import os
import pickle

def restaurar_query_encoded_de_pickle(pasta_resultado_caderno):
    # Lista para armazenar os dicionários lidos dos arquivos .pickle
    query_encoded_restaurado = []

    # Listando todos os arquivos .pickle no diretório especificado
    arquivos_pickle = [arq for arq in os.listdir(pasta_resultado_caderno) if arq.endswith('.pickle')]

    # Ordenando os arquivos pelo número (assumindo que os nomes dos arquivos seguem o padrão embeddings_query_X.pickle)
    arquivos_pickle.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

    # Lendo cada arquivo .pickle e restaurando o dicionário
    for nome_arquivo in arquivos_pickle:
        caminho_arquivo = os.path.join(pasta_resultado_caderno, nome_arquivo)
        with open(caminho_arquivo, 'rb') as arquivo_pickle:
            dicionario_restaurado = pickle.load(arquivo_pickle)
            query_encoded_restaurado.append(dicionario_restaurado)

    return reconstruir_dicionario_a_partir_de_lotes(query_encoded_restaurado)

In [39]:
query_encoded_restaurado = restaurar_query_encoded_de_pickle(PASTA_RESULTADO_CADERNO)

In [40]:
import numpy as np

query_hidden = query_encoded.copy()
query_hidden['cls_hidden_state'] = query_encoded_restaurado['cls_hidden_state']
query_hidden['mean_hidden_state'] = query_encoded_restaurado['mean_hidden_state']
print(f"cls_hidden_state: {query_hidden['cls_hidden_state'].size()}\n")
print(f"mean_hidden_state: {query_hidden['mean_hidden_state'].size()}")

cls_hidden_state: torch.Size([150, 1024])

mean_hidden_state: torch.Size([150, 1024])


In [42]:
# Extraindo os embeddings de duas queries
embedding1_tensor = query_hidden['mean_hidden_state'][0]
embedding2_tensor = query_hidden['mean_hidden_state'][4]

# Normalizando os embeddings
embedding1_norm = embedding1_tensor / embedding1_tensor.norm()
embedding2_norm = embedding2_tensor / embedding2_tensor.norm()

# Calculando a similaridade por cosseno
cosine_similarity = torch.dot(embedding1_norm, embedding2_norm)

print(f"Query 1: {query_hidden['TEXT'][0]}")
print(f"Query 2: {query_hidden['TEXT'][4]}")
print(f"Similaridade por cosseno: {cosine_similarity.item()}")

Query 1: técnica e preço
Query 2: sobrepreço e superfaturamento
Similaridade por cosseno: 0.36566948890686035


In [43]:
query_encoded_restaurado

{'key': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150],
 'cls_hidden_state': tensor([[-1.1093, -0.6649, -0

In [44]:
print(query_hidden['TEXT'][0])
print(query_hidden['TEXT'][1])
print(query_hidden['TEXT'][4])

técnica e preço
restos a pagar
sobrepreço e superfaturamento
